## USGS DOI Tool Python Module Examples
The USGS Digitial Object Identifier tool mints unique indentifers, it's important to be mindful of any development testing so large amounts of DOI's are not accidentally created or set to published. The USGS DOI Tool staging environment can help you as you develop a workflow. Any DOI that is created there is assigned a 10.5072 DOI where they will never be sent to DataCite to mint. 

In [ ]:
import os
import time
import json
import requests
import getpass

from usgs_datatools import doi
from usgs_datatools.doi import add_primary_doi_manager

### Configuration
The tool expects a valid USGS Active Directory account formatted as, username@usgs.gov. 

In [ ]:
doi_p = 'doi:10.5066/F7SB43S8'  # Sample production DOI my user has access to
doi_s = 'doi:10.5072/FK2J38SV7D'  # Sample staging DOI my user has access to

username = 'jsmith@usgs.gov'
password = getpass.getpass('USGS AD Password: ')

### Create DOI Tool Session
This is where you can specify your DOI Tool environment, (production and staging).

In [ ]:
#doi_session = doi.DoiSession()  # Production
DoiSession = doi.DoiSession(env='staging')  # Staging

### Authentication

In [ ]:
DoiSession.doi_authenticate(username, password)

### Fetch DOI Attributes 
A quick note, the "get_doi" function doesn't always fetch the correct status so in the scenario you are editing a reserved (on hold) DOI and want it to remain reserved (on hold - not public) please change the status value to "reserved". Otherwise this will default to public

In [ ]:
# Get doi
sample_doi = DoiSession.get_doi(doi_s)

# Change DOI attributes. 
sample_doi['status'] = 'reserved'
sample_doi['title'] = 'SERVICE TEST ITEM ' + cur_time
sample_doi  # Display to verify.

### Update Existing DOI

In [ ]:
test_update = DoiSession.doi_update(sample_doi)

### Test if DOI Updated

In [ ]:
did_it_update = DoiSession.get_doi(sample_doi['identifier'])
did_it_update

### Create New DOI
The same application rules are still in place here. Keep in mind, DOIs cannot have the same resourceURL as another. 

In [ ]:
# Get the sample doi and make slight updates. 
did_it_update['title'] = 'TEST Create New ' 
did_it_update['identifier'] = ''
did_it_update['status'] = 'reserved'
did_it_update['resourceURL'] = ''

# Add additional DOI Tool manager (primary). 
add_primary_doi_manager(did_it_update, 'janesmith@usgs.gov')

# Create new doi
DoiSession.doi_create(did_it_update)

#### References

[DOI Tool Staging](https://www1-staging.snafu.cr.usgs.gov/csas/doi/)

[DOI Tool Production](https://www1.usgs.gov/csas/doi/)